# Hello, FABRIC: Create your first FABRIC slice


## Setup the Experiment

#### Import the FABRIC API

In [1]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
Bastion Host,bastion.fabric-testbed.net
Bastion Username,tsai_0000103277
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key_file_name
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
Bastion Host,bastion.fabric-testbed.net
Bastion Username,tsai_0000103277
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key_file_name
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


#### Create the FABRIC Proxies

The FABRIC API is used via proxy objects that manage connections to the control framework.  

#### (Optional) Query Available Resources

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [2]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
except Exception as e:
    print(f"Error: {e}")

Available Resources: Name      PTP Capable      CPUs  Cores        Ram (G)    Disk (G)    Basic (100 Gbps NIC)    P4-Switch    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)    A30 (GPU)    A40 (GPU)    FPGA-Xilinx-U280    FPGA-Xilinx-SN1022
--------  -------------  ------  -----------  ---------  ----------  ----------------------  -----------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------  -----------  -----------  ------------------  --------------------
CERN      True               12  328/768      1776/2868  0/0         0/0                     0/0          0/0                             0/0                            0/0                 0/0               0/0              0/0          0/0          0/0                 0/0
HAWI      True               10  348/640      1030/2390  0/0         0/0                     0/0          0/0                

## Create the Experiment Slice

#### Configure the Experiment Parameters



### Create Slice

<img src="./figs/SingleNode.png" width="20%"><br>

Create a single node with basic compute capabilities. The submit function will block until the node is ready and will display a progress bar.


In [20]:
slice.delete()

In [21]:
# Configuration variables
slice_name = 'K8s_on_FABRIC'
site = "TOKY"
image = 'default_ubuntu_20'
network_name = 'NET1'

# Create lists for node configurations
nic_names, node_names = [], []
for i in range(1, 3):
    if i == 1:
        node_names.append(f"cpnode")
        nic_names.append(f"NIC1")
    else:
        node_names.append(f"wknode{i-1}")
        nic_names.append(f"NIC{i}")

print(f"Site: {site}")

try:
    # Create Slice
    slice = fablib.new_slice(slice_name)
    
    # Add nodes and interfaces
    nodes = []
    ifaces = []
    for i in range(len(node_names)):
        node = slice.add_node(
            name=node_names[i],
            site=site,
            image=image
        )
        nodes.append(node)
        iface = node.add_component(
            model='NIC_Basic', 
            name=nic_names[i]
        ).get_interfaces()[0]
        ifaces.append(iface)
    
    # Network
    net1 = slice.add_l3network(
        name=network_name, 
        interfaces=ifaces, 
        type='IPv4'
    )
    
    # Submit Slice Request
    slice_id = slice.submit()

except Exception as e:
    print(f"{e}")


Retry: 16, Time: 495 sec


ID,f2875b33-26fc-44e3-8d0a-cd1f86ad4ef2
Name,MySlice
Lease Expiration (UTC),2024-11-13 03:16:47 +0000
Lease Start (UTC),2024-11-12 03:16:47 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
84f7a01d-faed-484d-b1de-aacc66ea641a,Node1,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.201,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.201,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
946b7e1a-f9f1-4ecb-8dd9-fbd1a004a7ef,Node10,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.238,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.238,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
264b8994-456c-41c4-8da0-d81bf548ce67,Node2,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.16,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.16,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c2f832c3-548c-41ff-b448-3a78bc788847,Node3,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.41,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.41,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
3df5e63c-1fd5-41af-a718-3448f655cf3e,Node4,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.194,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.194,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c9487e6b-307b-40a6-8b2c-ae701b0cd79e,Node5,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.141,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.141,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2638264b-18ff-416e-98b1-a9c0752ebdbc,Node6,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.107,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.107,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
dd42dae6-272e-4a9d-9d98-b41f367f5d3b,Node7,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.82,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.82,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2b1ae31f-e6dd-437a-b93b-735b03bdf0cc,Node8,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.29,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.29,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
2c9ac2df-8ef1-44de-919a-a1aeee402f41,Node9,2,8,10,default_ubuntu_20,qcow2,toky-w3.fabric-testbed.net,TOKY,ubuntu,133.69.160.206,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.206,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
4d1b27ff-2544-46a8-8065-8bd612e3e5a1,NET1,L3,FABNetv4,TOKY,10.146.2.0/24,10.146.2.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-NIC1-p1,p1,Node1,NET1,100,config,,02:2D:08:2E:E2:8E,enp7s0,enp7s0,fe80::2d:8ff:fe2e:e28e,4,HundredGigE0/0/0/9
Node2-NIC2-p1,p1,Node2,NET1,100,config,,16:EC:1D:91:AA:06,enp7s0,enp7s0,fe80::14ec:1dff:fe91:aa06,4,HundredGigE0/0/0/9
Node3-NIC3-p1,p1,Node3,NET1,100,config,,1A:02:CC:B9:BD:5F,enp7s0,enp7s0,fe80::1802:ccff:feb9:bd5f,4,HundredGigE0/0/0/9
Node4-NIC4-p1,p1,Node4,NET1,100,config,,2E:87:2C:A6:F1:9D,enp7s0,enp7s0,fe80::2c87:2cff:fea6:f19d,4,HundredGigE0/0/0/9
Node5-NIC5-p1,p1,Node5,NET1,100,config,,32:76:0A:24:DB:BF,enp7s0,enp7s0,fe80::3076:aff:fe24:dbbf,4,HundredGigE0/0/0/9
Node6-NIC6-p1,p1,Node6,NET1,100,config,,2E:C8:C8:BA:94:15,enp7s0,enp7s0,fe80::2cc8:c8ff:feba:9415,4,HundredGigE0/0/0/9
Node7-NIC7-p1,p1,Node7,NET1,100,config,,32:47:DB:A5:C0:03,enp7s0,enp7s0,fe80::3047:dbff:fea5:c003,4,HundredGigE0/0/0/9
Node8-NIC8-p1,p1,Node8,NET1,100,config,,36:17:CC:8F:5C:6E,enp7s0,enp7s0,fe80::3417:ccff:fe8f:5c6e,4,HundredGigE0/0/0/9
Node9-NIC9-p1,p1,Node9,NET1,100,config,,36:D5:B1:68:BF:53,enp7s0,enp7s0,fe80::34d5:b1ff:fe68:bf53,4,HundredGigE0/0/0/9
Node10-NIC10-p1,p1,Node10,NET1,100,config,,36:51:ED:1E:D0:4E,enp7s0,enp7s0,fe80::3451:edff:fe1e:d04e,4,HundredGigE0/0/0/9



Time to print interfaces 543 seconds


### Print the Node's Attributes

Each node in the slice has a set of get functions that return the node's attributes.

In [22]:
try:
    slice = fablib.get_slice(slice_id=slice_id)
    nodes = slice.get_nodes()
    node_dict = {}
    address_dict = {}
    
    # Create node and address variables
    for i, node in enumerate(nodes, 1):
        node_name = node_names[i-1]
        node_dict[node_name] = node
        globals()[node_name] = node
        address_dict[f"{node_name}_address"] = None
        globals()[f"{node_name}_address"] = None
        print(f"{node_name}: {node}")

    # Get network info
    network = slice.get_network(name=network_name)
    network_available_ips = network.get_available_ips()
    print(f"Network: {network}")
    
    # Configure IPs for all nodes
    for node_name in node_names:
        node = node_dict[node_name]
        node_iface = node.get_interface(network_name=network_name)
        
        # Assign and store IP address
        node_address = network_available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_address, subnet=network.get_subnet())
        address_dict[f"{node_name}_address"] = node_address
        globals()[f"{node_name}_address"] = node_address
        
        # Print node config
        print(f"\nConfiguration for {node_name}:")
        print(f"Address assigned: {node_address}")
        stdout, _ = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        print(stdout)
        stdout, _ = node.execute('ip route list')
        print(stdout)

    # Print summary
    print("\nNode and Address Summary:")
    for node_name in node_names:
        print(f"{node_name}: {globals()[node_name]}")
        print(f"{node_name}_address: {globals()[f'{node_name}_address']}")

except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Available nodes:
node1: -----------------  --------------------------------------------------------------------------------------------------------------------
ID                 84f7a01d-faed-484d-b1de-aacc66ea641a
Name               Node1
Cores              2
RAM                8
Disk               10
Image              default_ubuntu_20
Image Type         qcow2
Host               toky-w3.fabric-testbed.net
Site               TOKY
Management IP      133.69.160.201
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@133.69.160.201
-----------------  --------------------------------------------------------------------------------------------------------------------
node2: -----------------  -------------------------------------------------------------------------------------------------------------------
ID                 264b8994-456c-41c4-8da0-d81bf548ce67
Name               


## Start the control plane
We follow the instructions that we have here: https://github.com/apache/openwhisk-deploy-kube/blob/master/docs/k8s-diy-ubuntu.md

In [23]:
try:
    file_attributes = cpnode.upload_file(local_file_path="config_control_plane.sh", remote_file_path="config_control_plane.sh")
    
    stdout, stderr = cpnode.execute(f"chmod +x config_control_plane.sh && ./config_control_plane.sh {network.get_subnet()} {cpnode_address}")

except Exception as e:
    print(f"Exception: {e}")
 



Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3303 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [483 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [14.3 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3247 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [456 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 c-n-f Metadata [548 B]
Get:12 http://security.ubuntu.com/ubuntu focal-se

## Start the worker node
Put the join command in the `join_cmd` variable. The command should look like this: ` kubeadm join <control-plane-ip>:6443 --token <token> --discovery-token-ca-cert-hash sha256:<hash>`

In [26]:
join_cmd = "sudo kubeadm join 10.146.4.2:6443 --token l255bp.wqi5i6br0jg7f4z2 --discovery-token-ca-cert-hash sha256:069646097e377bcd9e6d66ee7779a0d3e0930d95d1e6a79f73f22f70b1098b5e"

try:
    # Loop through worker nodes in node_dict
    for node_name, node in node_dict.items():
        if node_name == 'cpnode':  # Skip control plane node
            continue
            
        print(f"\nConfiguring {node_name}...")
        
        # Upload and execute config script
        try:
            print(f"Uploading and executing config_worker_node.sh on {node_name}...")
            file_attributes = node.upload_file(
                local_file_path="config_worker_node.sh", 
                remote_file_path="config_worker_node.sh"
            )
            exec_cmd = f"chmod +x config_worker_node.sh && ./config_worker_node.sh {join_cmd}"
            stdout, stderr = node.execute(exec_cmd)
            print(f"Config output for {node_name}:", stdout)
        except Exception as e:
            print(f"Failed to configure {node_name}: {e}")
            continue  # Skip to next node if configuration fails

except Exception as e:
    print(f"Main exception: {e}")
    traceback.print_exc()


Configuring worker node2...
Uploading and executing config_worker_node.sh on node2...


Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3303 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse Translation-en [104 kB]
Get:11 http://nova.clouds.archive.ubuntu.co

## Monitor the kubernetes cluster

In [27]:
try:
    # Get all resources across namespaces
    print("Getting all kubernetes resources...")
    stdout, stderr = cpnode.execute("kubectl get all --all-namespaces")
    print(f"All resources:\n{stdout}")
    if stderr:
        print(f"Stderr: {stderr}")
    
    # Get all pods with more details
    print("\nGetting detailed pods status across all namespaces...")
    stdout, stderr = cpnode.execute("kubectl get pods -A -o wide")
    print(f"Detailed pods status:\n{stdout}")
    if stderr:
        print(f"Stderr: {stderr}")
        
    # Describe kube-system namespace pods
    print("\nDescribing kube-system pods...")
    stdout, stderr = cpnode.execute("kubectl describe pods -n kube-system")
    print(f"Kube-system pods details:\n{stdout}")
    if stderr:
        print(f"Stderr: {stderr}")
    
    # Get logs from specific system pods
    system_components = ['kube-apiserver', 'kube-controller-manager', 'kube-scheduler', 'etcd']
    
    for component in system_components:
        print(f"\nGetting logs for {component}...")
        try:
            # First get the pod name
            cmd = f"kubectl get pods -n kube-system -l component={component} -o jsonpath='{{.items[0].metadata.name}}'"
            stdout, stderr = cpnode.execute(cmd)
            if stdout:
                pod_name = stdout.strip()
                # Then get the logs
                stdout, stderr = cpnode.execute(f"kubectl logs -n kube-system {pod_name} --tail=50")
                print(f"Logs from {component} ({pod_name}):\n{stdout}")
            else:
                print(f"No pod found for component {component}")
        except Exception as e:
            print(f"Error getting logs for {component}: {e}")

except Exception as e:
    print(f"Exception while monitoring kubernetes cluster: {e}")
    traceback.print_exc()



NAME     STATUS     ROLES           AGE     VERSION
node1    Ready      control-plane   16m     v1.31.2
node10   NotReady   <none>          2s      v1.31.2
node2    Ready      <none>          13m     v1.31.2
node3    Ready      <none>          12m     v1.31.2
node4    Ready      <none>          10m     v1.31.2
node5    Ready      <none>          8m37s   v1.31.2
node6    Ready      <none>          6m55s   v1.31.2
node7    Ready      <none>          5m4s    v1.31.2
node8    Ready      <none>          3m22s   v1.31.2
node9    Ready      <none>          102s    v1.31.2
stdout: NAME     STATUS     ROLES           AGE     VERSION
node1    Ready      control-plane   16m     v1.31.2
node10   NotReady   <none>          2s      v1.31.2
node2    Ready      <none>          13m     v1.31.2
node3    Ready      <none>          12m     v1.31.2
node4    Ready      <none>          10m     v1.31.2
node5    Ready      <none>          8m37s   v1.31.2
node6    Ready      <none>          6m55s   v1.31.2
node

## Node has successfully joined the cluster.

# Deploying a hello world application.

## First, we pull a hello world image and create a "deployment".

In [28]:
try:
    stdout, stderr = cpnode.execute("kubectl create deployment kubernetes-bootcamp --image=gcr.io/google-samples/kubernetes-bootcamp:v1 --replicas=9")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
 

deployment.apps/kubernetes-bootcamp created
stdout: deployment.apps/kubernetes-bootcamp created

stderr: 


## Now let's do some status commands.

In [134]:
try:
    stdout, stderr = cpnode.execute("kubectl get pods --all-namespaces")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

NAMESPACE     NAME                                       READY   STATUS    RESTARTS   AGE
default       kubernetes-bootcamp-68cfbdbb99-w2pk4       1/1     Running   0          14s
kube-system   calico-kube-controllers-6879d4fcdc-rrp2b   1/1     Running   0          6m40s
kube-system   calico-node-kjxnc                          1/1     Running   0          3m44s
kube-system   calico-node-xrzxv                          1/1     Running   0          6m40s
kube-system   coredns-7c65d6cfc9-lsqw4                   1/1     Running   0          6m40s
kube-system   coredns-7c65d6cfc9-tqzzx                   1/1     Running   0          6m40s
kube-system   etcd-node1                                 1/1     Running   0          6m47s
kube-system   kube-apiserver-node1                       1/1     Running   0          6m47s
kube-system   kube-controller-manager-node1              1/1     Running   0          6m47s
kube-system   kube-proxy-472xw                           1/1     Running   0        

## Pick the correct pod name and run the command below.

In [29]:

try:
    stdout, stderr = cpnode.execute("kubectl describe pod kubernetes-bootcamp-68cfbdbb99-d67sb")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

Name:             kubernetes-bootcamp-68cfbdbb99-d67sb
Namespace:        default
Priority:         0
Service Account:  default
Node:             node3/10.20.5.91
Start Time:       Tue, 12 Nov 2024 03:47:12 +0000
Labels:           app=kubernetes-bootcamp
                  pod-template-hash=68cfbdbb99
Annotations:      cni.projectcalico.org/containerID: 06ee87410189c92c674d921c8a11167abab926f81d6f74da7c115f909272d113
                  cni.projectcalico.org/podIP: 10.146.2.113/32
                  cni.projectcalico.org/podIPs: 10.146.2.113/32
Status:           Running
IP:               10.146.2.113
IPs:
  IP:           10.146.2.113
Controlled By:  ReplicaSet/kubernetes-bootcamp-68cfbdbb99
Containers:
  kubernetes-bootcamp:
    Container ID:   containerd://5c233800d4a748ec92341a274e4d2e158872d10d18e214ac97d67c12a27119f3
    Image:          gcr.io/google-samples/kubernetes-bootcamp:v1
    Image ID:       gcr.io/google-samples/kubernetes-bootcamp@sha256:0d6b8ee63bb57c5f5b6156f446b3bc3b3c143d

## The next thing we need to do is to create what is called a "service".

We are going to use it to expose the deployment to the outside, through a port, which is 8080. Like this:

Note that the service itself will still need to be exposed. There's another "expose" step that we need to make.

In [30]:
try:
    stdout, stderr = cpnode.execute('kubectl expose deployment/kubernetes-bootcamp --type="ClusterIP" --port 8080')
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

service/kubernetes-bootcamp exposed
stdout: service/kubernetes-bootcamp exposed

stderr: 


## Let's check if the service was created.

In [31]:
try:
    stdout, stderr = cpnode.execute("kubectl get service kubernetes-bootcamp")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

NAME                  TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
kubernetes-bootcamp   ClusterIP   10.111.180.18   <none>        8080/TCP   11s
stdout: NAME                  TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
kubernetes-bootcamp   ClusterIP   10.111.180.18   <none>        8080/TCP   11s

stderr: 


## Finally, we need to run a port forwarding command in order to expose the service to the outside.

Modify the --address flag. Use the "CLUSTER-IP" that is output form the command above.

In [32]:
try:
    stdout, stderr = cpnode.execute("kubectl port-forward --address 10.111.180.18 service/kubernetes-bootcamp 8080:8080 > /dev/null 2>&1 &")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

stdout: 
stderr: 


## Now our application should finally be visible. Let's test the deployment on the control plane machine itself.

In [35]:
try:
    stdout, stderr = cpnode.execute("curl 10.111.180.18:8080")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-68cfbdbb99-bdrtr | v=1
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    84    0    84    0     0  14000      0 --:--:-- --:--:-- --:--:-- 14000
 stdout: Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-68cfbdbb99-bdrtr | v=1

stderr:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    84    0    84    0     0  14000      0 --:--:-- --:--:-- --:--:-- 14000



## Delete Slice

Please delete your slicd when you are done with your experiment.

In [2]:
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")

Fail: name 'slice_name' is not defined
